In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/genaik2025-aylin/results_clean_v2.txt
/kaggle/input/genaik2025-aylin/summary_clean_v2.txt
/kaggle/input/genaik2025-aylin/cypher_query_v2.txt
/kaggle/input/genaik2025-aylin/summary_clean.txt
/kaggle/input/genaik2025-aylin/results_clean.txt
/kaggle/input/genaik2025-aylin/cypher_query.txt
/kaggle/input/genaik2025-aylin/neo4j_query_table_data_2024-10-12.csv
/kaggle/input/genaik2025-aylin/Geoffrey_Hinton_KG.csv


# Sources:
- https://github.com/sunnysavita10/Generative-AI-Indepth-Basic-to-Advance/blob/main/RAG%20with%20Knowledge%20Graph%20Neo4j/RAG_With_Knowledge_graph(Neo4j).ipynb
- https://www.youtube.com/watch?v=9ZBamhWPSKg

In [4]:
!pip install --upgrade --quiet langchain_groq
!pip install --upgrade --quiet sentence-transformers
!pip install --upgrade --quiet  langchain langchain-community  langchain-experimental neo4j 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.3 requires cubinlinker, which is not installed.
cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires ptxcompiler, which is not installed.
cuml 24.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 24.8.3 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.8.3 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.6.0 which is incompatible.
distributed 2024.7.1 requires dask==2024.7.1, but you have dask 2024.9.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 24.1 which is incompatible.
jupyterlab 4.2.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which 

In [5]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph
from getpass import getpass
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough


/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
neo4j_url = getpass("NEO4J_CONNECTION_URL") 
neo4j_user = getpass("NEO4J_USER") 
neo4j_password = getpass("NEO4J_PASSWORD") 
graph = Neo4jGraph(neo4j_url, neo4j_user, neo4j_password)

NEO4J_CONNECTION_URL ·········································
NEO4J_USER ·····
NEO4J_PASSWORD ···········································


In [7]:
if "GROQ_API_KEY" not in os.environ: 
    os.environ["GROQ_API_KEY"] = getpass("Enter your Groq API key: ")

Enter your Groq API key:  ························································


In [8]:
model_name = "sentence-transformers/all-MiniLM-L12-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/tmp/ipykernel_30/2407877103.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  hf = HuggingFaceEmbeddings(
/opt/conda/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
os.environ["NEO4J_URI"] = neo4j_url
os.environ["NEO4J_USERNAME"] = neo4j_user
os.environ["NEO4J_PASSWORD"] = neo4j_password

In [10]:
vector_index = Neo4jVector.from_existing_graph(
    hf,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [11]:
vector_index

In [12]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [13]:
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the award, birthPlace, characteristics, date, degree, institution, invention, nationality, person, relation, role and important entities that "
        "appear in the text",
    )

In [14]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting oaward, birthPlace, characteristics, date, degree, institution, invention, nationality, person, relation, role and important entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)
   

In [15]:
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192")

In [16]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [17]:
entity_chain.invoke({"question": "Where was Geoffrey Hinton born?"}).names

['Geoffrey Hinton']

In [18]:
def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search quesry for a given input string.
    
    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similiarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

In [25]:
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [24]:
graph.refresh_schema()
print(graph.schema)

Node properties:
birthPlace {id: STRING, entityId: INTEGER}
person {id: STRING, entityId: INTEGER}
institution {id: STRING, entityId: INTEGER}
role {id: STRING, entityId: INTEGER}
award {id: STRING, entityId: INTEGER}
nationality {id: STRING, entityId: INTEGER}
invention {id: STRING, entityId: INTEGER}
characteristics {id: STRING, entityId: INTEGER}
date {id: STRING, entityId: INTEGER}
relation {id: STRING, entityId: INTEGER}
degree {id: STRING, entityId: INTEGER}
Relationship properties:

The relationships:
(:person)-[:bornIn]->(:birthPlace)
(:person)-[:earnedDegreeFrom]->(:institution)
(:person)-[:earnedDegree]->(:degree)
(:person)-[:worksAt]->(:institution)
(:person)-[:developed]->(:invention)
(:person)-[:collaboratedWith]->(:person)
(:person)-[:collaboratedWith]->(:institution)
(:person)-[:hasAward]->(:award)
(:person)-[:bornAt]->(:date)
(:person)-[:hasNationality]->(:nationality)
(:person)-[:hasRole]->(:role)
(:person)-[:coFounded]->(:institution)
(:person)-[:coInventing]->(:inven

In [26]:
print(structured_retriever("Where was Geoffrey Hinton born"))

Geoffrey Hinton - bornIn -> London
Geoffrey Hinton - worksAt -> University College London
Geoffrey Hinton - worksAt -> Carnegie Mellon University
Geoffrey Hinton - worksAt -> University of Toronto
Geoffrey Hinton - worksAt -> Vector Institute
Geoffrey Hinton - worksAt -> Google Brain
Geoffrey Hinton - worksAt -> University of California, San Diego
Geoffrey Hinton - hasRole -> pursuit of knowledge and innovation
Geoffrey Hinton - hasRole -> advisory roles
Geoffrey Hinton - hasRole -> research
Geoffrey Hinton - hasRole -> cognitive psychologist and computer scientist
Geoffrey Hinton - hasRole -> advisory role
Geoffrey Hinton - coAuthored -> Ronald J. Williams
Geoffrey Hinton - coAuthored -> David Rumelhart
Geoffrey Hinton - developed -> mixtures of experts
Geoffrey Hinton - developed -> neural networks
Geoffrey Hinton - developed -> distributed representations
Geoffrey Hinton - developed -> time delay neural networks
Geoffrey Hinton - developed -> AI applications
Geoffrey Hinton - develo

In [27]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [28]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [29]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [30]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [32]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatGroq(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [33]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [34]:
prompt = ChatPromptTemplate.from_template(template)

In [35]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [36]:
chain.invoke({"question": "Which company or institution did Geoffrey Hintion founded?"})

Search query: Which company or institution did Geoffrey Hintion founded?


'Geoffrey Hinton co-founded the Vector Institute.'

In [37]:
chain.invoke(
    {
        "question": "When was he born?",
        "chat_history": [("Which company or institution did Geoffrey Hintion founded?", "Geoffrey Hinton co-founded the Vector Institute.")],
    }
)

Search query: When was Geoffrey Hinton born?


'Geoffrey Hinton was born on December 6, 1947.'